In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import OrderedDict 
%matplotlib inline 

#### Reading dataset

In [4]:
loc = './'
mapping=pd.read_csv(loc + 'comments_employee_mapping.csv').dropna()
likes=pd.read_csv(loc + 'comments_likeability.csv').dropna()
attr=pd.read_csv(loc + 'employee_attrition.csv').dropna()
hap=pd.read_csv(loc + 'happiness_level.csv').dropna()

attr['lastParticipationDate']=pd.to_datetime(attr['lastParticipationDate'],infer_datetime_format=True)
mapping['commentDate']=pd.to_datetime(mapping['commentDate'],infer_datetime_format=True)
hap['voteDate']=pd.to_datetime(hap['voteDate'],infer_datetime_format=True)

for d in [mapping,likes,attr,hap]:
    d['id']=d['employee'].map(str)+'_' + d['companyAlias']

/Users/apple/anaconda3/lib/python3.6/site-packages/dateutil/parser/_parser.py:1204: UnknownTimezoneWarning: tzname CET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
/Users/apple/anaconda3/lib/python3.6/site-packages/dateutil/parser/_parser.py:1204: UnknownTimezoneWarning: tzname CEST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


#### Mapping employee and comment id to range of integers 

In [5]:
employee_ids = attr['id'].values
init = np.arange(1, len(employee_ids) + 1)
map_employee = OrderedDict(list(zip(employee_ids, init)))

comment_ids = mapping['commentId'].values
init = np.arange(1, len(comment_ids) + 1)
map_comment = OrderedDict(list(zip(comment_ids, init)))

print(len(map_employee), len(np.unique(employee_ids)))
print(len(map_comment), len(np.unique(comment_ids)))

4377 4377
38993 38993


#### Building employee - comment reaction matrix

In [6]:
d = np.zeros([len(employee_ids),len(comment_ids)])

for i in range(len(employee_ids)):
    emp_idx = map_employee[likes['id'][i]]
    com_idx = map_comment[likes['commentId'][i]]
    if likes['liked'][i]==True:
        d[emp_idx, com_idx] = 1
    else:
        d[emp_idx, com_idx] = -1